In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# > COVID-19 Vaccine South America Exploratory Data Analysis
This Project shows an exploratory data analysis of the Covid-19 vaccine dataset. The analysis is divided in 3 parts:
1. First is a Worldwide analysis
2. Second is South America Analysis
3. Third we will use UN data to find correlations and see if there are any relationship between United Nations data and the vaccination's speed

> **First Part**
Cleaning and Analyze the Data

In [ ]:
#Importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#importing dataset and a brief summary
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
df.head()

                 

In [ ]:
df.info()

In [ ]:
#Chaging the Nan value
df.fillna(0,inplace = True)

### Vaccines - Country  Strategy
Note: If a country uses more than one vaccine, the graph will show  both vaccines in  one bar

In [ ]:
df_vaccines = df.drop_duplicates(subset=['country','vaccines'], keep ='last')
vacinas_dictionary = dict(df_vaccines.groupby('vaccines')['country'].count())
df_vaccines.groupby('vaccines')['country'].count().plot(kind='bar')


In [ ]:
df_vaccines.groupby('vaccines')['country'].count().plot(kind='pie')

### Countries that most vaccinated

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
df.groupby(['country']).max()['people_vaccinated'].nlargest(10).plot.bar()
plt.title('Top 10 countries that most vaccinated')
plt.xlabel('Countries')

### Countries that most vaccinated per hundred 

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
df.groupby(['country']).max()['people_vaccinated_per_hundred'].nlargest(10).plot.bar()
plt.title('Top 10 countries that most vaccinated per hundred')
plt.xlabel('Countries')

Looking  the graphs, Althoug USA and China has vaccinated more people that the other countrie, those populations are too big and they haven't vaccinated soo much citizens. Gibraltar and Israel vaccinated almost 40% of their population

## South America Dataset
Now we wil investigate the speed of vaccination in South America 

In [ ]:
#South America Dataset
countries_sa = ['Brazil','Argentina','Chile','Venezuela','Bolivia','Colombia','Ecuador','Peru','Paraguay','Uruguay','Suriname','Guyana'] 
south_america_ds = df[df.country.isin(countries_sa)]
south_america_ds

### Looking for  total vaccination and total vaccination per hundred in the last day when data was collected

In [ ]:
#Finding which country vacinated more pepole
south_america_ds.groupby('country').total_vaccinations.max()

In [ ]:
#People vaccinated per hundre
south_america_ds.groupby('country').people_vaccinated_per_hundred.max()

### Evolution of People Vaccinated and people Vaccinated per hundred in South America

In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
south_america_ds.groupby(['date','country']).sum()['people_vaccinated'].unstack().plot(ax=ax)
plt.title('People Vaccinated')
plt.xlabel('Date')


In [ ]:
fig,ax = plt.subplots(figsize=(15,7))
south_america_ds.groupby(['date','country']).sum()['people_vaccinated_per_hundred'].unstack().plot(ax=ax)
plt.title('People Vaccinated per Hundred')
plt.xlabel('Date')


## Comparing vaccination speed with United Nations Data

In this part we will compare tha data of vaccination with some basic data about the countries. There is some relationship between countries GDP and vaccination's speed? And Health expenditure?

In [ ]:
#Adding Undataset
undt = pd.read_csv('../input/undata-country-profiles/country_profile_variables.csv')
undt.head()

In [ ]:
#Creating the UN datasey with South america countries
saundt = undt[undt.country.isin(countries_sa)]
saundt

In [ ]:
#Finding correlations
new_df = df.sort_values('people_vaccinated_per_hundred', ascending=False).drop_duplicates('country').sort_index().reset_index(drop=True)
new_df

In [ ]:
lists_countries_vaccine = list(new_df['country'])
dfun_new = undt[undt['country'].isin(lists_countries_vaccine)]

dfun_new

In [ ]:
lists_countries_vaccine_2 = list(dfun_new['country'])
new_df = new_df[new_df['country'].isin(lists_countries_vaccine_2)]
new_df

In [ ]:
dfun_new.reset_index(drop=True, inplace=True)
new_df.reset_index(drop=True, inplace=True)
df3 = pd.concat([dfun_new.iloc[:,3],dfun_new.iloc[:,6:8],dfun_new.iloc[:,31:33],new_df.iloc[:,4:9]], axis = 1)
df3


In [ ]:
for x in range(4):
    df3.iloc[:,x] = df3.iloc[:,x].astype(float)


#for x in range(3):
    #df3.iloc[:,(x+7)] = df3.iloc[:,(x+7)].astype(float)

In [ ]:
sns.heatmap(df3.corr(), annot= True)

As we can see in the heatmap, only GDP has a big correlation with people vaccinated. Correlations between people vaccinated per hundred and the population in the country shows a slowly negative correlation.It means that countries with few people vaccinated their population faster than other countries. Now i will investigate this